## First Dataset

In [51]:
import pandas as pd
import numpy as np

In [52]:
df = pd.read_csv("BL-Flickr-Images-Book.csv")
df.head(3)

,Identifier,Edition Statement,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Corporate Author,Corporate Contributors,Former owner,Engraver,Issuance type,Flickr URL,Shelfmarks
0,206,NaN,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12641.b.30.
1,216,NaN,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12626.cc.2.
2,218,NaN,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",NaN,NaN,NaN,NaN,monographic,http://www.flickr.com/photos/britishlibrary/ta...,British Library HMNTS 12625.dd.1.


In [53]:
df.columns

Index(['Identifier', 'Edition Statement', 'Place of Publication',
       'Date of Publication', 'Publisher', 'Title', 'Author', 'Contributors',
       'Corporate Author', 'Corporate Contributors', 'Former owner',
       'Engraver', 'Issuance type', 'Flickr URL', 'Shelfmarks'],
      dtype='object')

In [54]:
print("Shape of the dataframe: {}".format(df.shape))

Shape of the dataframe: (8287, 15)


Let's see all the missing values for each column

We look there are columns that all the values have missing data and we can see that a handful of columns provide ancillary information that would be helpful to the library but is not very descriptive of the book themselves

In [55]:
drop_columns = ["Edition Statement", "Corporate Author", "Corporate Contributors", "Former owner", "Engraver", "Issuance type", "Shelfmarks"]
df.drop(columns = drop_columns, axis = 1, inplace = True)
df.head()

,Identifier,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Flickr URL
0,206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",http://www.flickr.com/photos/britishlibrary/ta...
1,216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
2,218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
3,472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",http://www.flickr.com/photos/britishlibrary/ta...
4,480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",http://www.flickr.com/photos/britishlibrary/ta...


**Changing the index of a dataframe**

In [56]:
df["Identifier"].is_unique

True

Let's replace the existing index with the column "Identifier" using *set_index*

In [57]:
df.set_index("Identifier", inplace = True)
df.head()

,Place of Publication,Date of Publication,Publisher,Title,Author,Contributors,Flickr URL
Identifier,,,,,,,
206,London,1879 [1878],S. Tinsley & Co.,Walter Forbes. [A novel.] By A. A,A. A.,"FORBES, Walter.",http://www.flickr.com/photos/britishlibrary/ta...
216,London; Virtue & Yorston,1868,Virtue & Co.,All for Greed. [A novel. The dedication signed...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
218,London,1869,"Bradbury, Evans & Co.",Love the Avenger. By the author of “All for Gr...,"A., A. A.","BLAZE DE BURY, Marie Pauline Rose - Baroness",http://www.flickr.com/photos/britishlibrary/ta...
472,London,1851,James Darling,"Welsh Sketches, chiefly ecclesiastical, to the...","A., E. S.","Appleyard, Ernest Silvanus.",http://www.flickr.com/photos/britishlibrary/ta...
480,London,1857,Wertheim & Macintosh,"[The World in which I live, and my place in it...","A., E. S.","BROOME, John Henry.",http://www.flickr.com/photos/britishlibrary/ta...


In [58]:
df.loc[206]

Place of Publication                                               London
Date of Publication                                           1879 [1878]
Publisher                                                S. Tinsley & Co.
Title                                   Walter Forbes. [A novel.] By A. A
Author                                                              A. A.
Contributors                                              FORBES, Walter.
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 206, dtype: object

**Tidying up Fields in the Data**

In [59]:
df.isnull().sum()

Place of Publication       0
Date of Publication      181
Publisher               4195
Title                      0
Author                  1778
Contributors               0
Flickr URL                 0
dtype: int64

Let's clean the columns "Data of Publication" & "Place of publication"

In [60]:
df.dtypes

Place of Publication    object
Date of Publication     object
Publisher               object
Title                   object
Author                  object
Contributors            object
Flickr URL              object
dtype: object

One field where it makes sense to enforce a numeric value is the "Date of publication" so that we can do calculations down the road:

In [61]:
df.loc[1905:, "Date of Publication"].head(10)

Identifier
1905           1888
1929    1839, 38-54
2836           1897
2854           1865
2956        1860-63
2957           1873
3017           1866
3131           1899
4598           1814
4884           1820
Name: Date of Publication, dtype: object

A particular book can have only one date of publication. Therefore, we need to do the following:

* Remove the extra dates in square brackets, wherever present: 1879 [1878]
* Convert date ranges to their “start date”, wherever present: 1860-63; 1839, 38-54
* Completely remove the dates we are not certain about and replace them with NumPy’s NaN: [1897?]
* Convert the string nan to NumPy’s NaN value

regular expression to extract the publication year

In [62]:
regex = r"^(\d{4})"

In [63]:
extr = df["Date of Publication"].str.extract(regex, expand = False)
extr.head()

Identifier
206    1879
216    1868
218    1869
472    1851
480    1857
Name: Date of Publication, dtype: object

We can get it's easily numerical values

In [64]:
df["Date of Publication"] = pd.to_numeric(extr)
df["Date of Publication"].dtypes

dtype('float64')

In [65]:
df["Date of Publication"].isnull().sum() / len(df)

0.11717147339205986

This results in about one in every 10% values being missing, which is a small price to pay for now being able to do computations on the remaining valid values

**Combining str Methods with numpy to clean columns**

To clean the Place of Publication field, we can combine Pandas str methods with NumPy’s np.where function, which is basically a vectorized form of Excel’s IF() macro. It has the following syntax:

In [66]:
df["Place of Publication"].head(10)

Identifier
206                                  London
216                London; Virtue & Yorston
218                                  London
472                                  London
480                                  London
481                                  London
519                                  London
667     pp. 40. G. Bryan & Co: Oxford, 1898
874                                 London]
1143                                 London
Name: Place of Publication, dtype: object

In [67]:
df.loc[4157862]

Place of Publication                                  Newcastle-upon-Tyne
Date of Publication                                                  1867
Publisher                                                      T. Fordyce
Title                   Local Records; or, Historical Register of rema...
Author                      FORDYCE, T. - Printer, of Newcastle-upon-Tyne
Contributors             SYKES, John - Bookseller, of Newcastle-upon-Tyne
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4157862, dtype: object

In [68]:
df.loc[4159587]

Place of Publication                                  Newcastle upon Tyne
Date of Publication                                                  1834
Publisher                                                Mackenzie & Dent
Title                   An historical, topographical and descriptive v...
Author                                              Mackenzie, E. (Eneas)
Contributors                                         ROSS, M. - of Durham
Flickr URL              http://www.flickr.com/photos/britishlibrary/ta...
Name: 4159587, dtype: object

These two books were published in the same place, but one has hyphens in the name of the place while the other does not.

To clean this column in one sweep, we can use str.contains() to get a Boolean mask.

In [69]:
pub = df["Place of Publication"]
london = pub.str.contains("London")
london[:5]

Identifier
206    True
216    True
218    True
472    True
480    True
Name: Place of Publication, dtype: bool

In [70]:
oxford = pub.str.contains("Oxford")

In [71]:
df["Place of Publication"] = np.where(london, "london", np.where(oxford, "oxford", pub.str.replace("-", " ")))
df["Place of Publication"].head()

Identifier
206    london
216    london
218    london
472    london
480    london
Name: Place of Publication, dtype: object

## Second Dataset

**Cleaning the Entire Dataset using the applymap Function**

There are some instances where it would be helpful to apply a customized function to each cell or element of a DataFrame. Pandas .applymap() method is similar to the in-built map() function and simply applies a function to all the elements in a DataFrame.

Let’s look at an example. We will create a DataFrame out of the “university_towns.txt” file:


In [72]:
name_file = "university_towns.txt"
university_towns = []

with open(name_file) as file:
  for line in file:
    if "[edit]" in line:
      state = line
    else:
      university_towns.append((state, line))

university_towns[:5]

[('Alabama[edit]\n', 'Auburn (Auburn University)[1]\n'),
 ('Alabama[edit]\n', 'Florence (University of North Alabama)\n'),
 ('Alabama[edit]\n', 'Jacksonville (Jacksonville State University)[2]\n'),
 ('Alabama[edit]\n', 'Livingston (University of West Alabama)[2]\n'),
 ('Alabama[edit]\n', 'Montevallo (University of Montevallo)[2]\n')]

We can wrap this list in a DataFrame and set the columns as “State” and “RegionName”.

In [74]:
towns_df = pd.DataFrame(university_towns, columns = ["State", "Region Name"])
towns_df.head()

,State,Region Name
0,Alabama[edit]\n,Auburn (Auburn University)[1]\n
1,Alabama[edit]\n,Florence (University of North Alabama)\n
2,Alabama[edit]\n,Jacksonville (Jacksonville State University)[2]\n
3,Alabama[edit]\n,Livingston (University of West Alabama)[2]\n
4,Alabama[edit]\n,Montevallo (University of Montevallo)[2]\n


In [75]:
def get_citystate(item):
  if " (" in item:
    return item[:item.find(" (")]
  elif "[" in item:
    return item[:item.find("[")]
  else:
    return item

In [76]:
towns_df = towns_df.applymap(get_citystate)
towns_df.head()

,State,Region Name
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo


First, we define a Python function that takes an element from the DataFrame as its parameter. Inside the function, checks are performed to determine whether there’s a ( or [ in the element or not.

Depending on the check, values are returned accordingly by the function. Finally, the applymap() function is called on our object. Now the DataFrame is much neater:

## Third Dataset

In this case, we’d want to rename columns and skip certain rows so that we can drill down to necessary information with correct and sensible labels.

In [77]:
df_olympics = pd.read_csv("olympics.csv")
df_olympics.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,NaN,? Summer,01 !,02 !,03 !,Total,? Winter,01 !,02 !,03 !,Total,? Games,01 !,02 !,03 !,Combined total
1,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
2,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
3,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
4,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12


This is messy indeed! The columns are the string form of integers indexed at 0. The row which should have been our header (i.e. the one to be used to set the column names) is at olympics_df.iloc[0]. This happened because our CSV file starts with 0, 1, 2, …, 15.

Therefore, we need to do two things:

Skip one row and set the header as the first (0-indexed) row
Rename the columns
We can skip rows and set the header while reading the CSV file by passing some parameters to the read_csv() function.

In [78]:
df_olympics = pd.read_csv("olympics.csv", header = 1)
df_olympics.head()

,Unnamed: 0,? Summer,01 !,02 !,03 !,Total,? Winter,01 !.1,02 !.1,03 !.1,Total.1,? Games,01 !.2,02 !.2,03 !.2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12


We now have the correct row set as the header and all unnecessary rows removed. Take note of how Pandas has changed the name of the column containing the name of the countries from NaN to Unnamed: 0.

To rename the columns, we will make use of a DataFrame’s rename() method, which allows you to relabel an axis based on a mapping (in this case, a dict).

In [79]:
df_olympics.columns

Index(['Unnamed: 0', '? Summer', '01 !', '02 !', '03 !', 'Total', '? Winter',
       '01 !.1', '02 !.1', '03 !.1', 'Total.1', '? Games', '01 !.2', '02 !.2',
       '03 !.2', 'Combined total'],
      dtype='object')

In [84]:
new_names = {"Unnamed: 0" : "Country", 
             '? Summer' : "Summer Olympics",
             '01 !' : "Gold",
             '02 !' : "Silver",
             '03 !' : "Bronze",
             "Total" : "Total",
             "? Winter" : "Winter Olympics",
             "01 !.1" : "Gold_1",
             "02 !.1" : "Silver_1",
             "03 !.1" : "Bronze_1",
             "Total.1" : "Total_1",
             "? Games" : "# Games",
             "01 !.2" : "Gold_2",
             "02 !.2" : "Silver_2",
             "03 !.2" : "Bronze_2",
             "Combined total" : "Combined total"}

In [85]:
df_olympics.rename(columns = new_names, inplace = True)
df_olympics.head()

,Country,Summer Olympics,Gold,Silver,Bronze,Total,Winter Olympics,Gold_1,Silver_1,Bronze_1,Total_1,# Games,Gold_2,Silver_2,Bronze_2,Combined total
0,Afghanistan (AFG),13,0,0,2,2,0,0,0,0,0,13,0,0,2,2
1,Algeria (ALG),12,5,2,8,15,3,0,0,0,0,15,5,2,8,15
2,Argentina (ARG),23,18,24,28,70,18,0,0,0,0,41,18,24,28,70
3,Armenia (ARM),5,1,2,9,12,6,0,0,0,0,11,1,2,9,12
4,Australasia (ANZ) [ANZ],2,3,4,5,12,0,0,0,0,0,2,3,4,5,12
